# Tf-IDF for Sponsored Legistlation
The goal is to generate characteristic words and phrases for one legislator's sponsored bills, to quickly give viewers a good idea of the legislator's work in Congress.

Steps:

1. Connect to either the relationshal Postgres DB or the Mongo DB, get the bill descriptions.

2. Great single documents for all legislators by appending all bill descriptions. 

3. Use sklearn, to calculate tf-idf for a ll 1/2/3 work combinations. 

4. Take the top 10 by tf-idf per legislator.

5. Create a new table in the postgres DB with these charwords. 

In [99]:
import numpy as np
import pandas as pd
import psycopg
from sqlalchemy import create_engine
import os
import dotenv
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

dotenv.load_dotenv()
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

### Connect the relational Postgres DB, get the bill description.

In [100]:
dbms = 'postgresql'
package = 'psycopg'
user = 'postgres'
password = POSTGRES_PASSWORD
host = 'localhost'
port = '5432'
db = 'contrans'

engine = create_engine(f"{dbms}+{package}://{user}:{password}@{host}:{port}/{db}")

In [101]:
myquery = """
SELECT b.bioguide_id, bv.text
FROM bill_versions bv
INNER JOIN bills b
    ON bv.bill_type = b.bill_type AND bv.bill_number = b.bill_number
"""

bill_description = pd.read_sql(myquery, con=engine)


In [102]:
bill_description

,bioguide_id,text
0,J000293,<p><strong>Shutdown Fairness Act</strong></p><...
1,O000173,<p><strong>Federal Worker Childcare Protection...
2,A000380,<p>This bill requires the federal government t...
3,S001194,<p><b>Federal Employees Civil Relief Act</b></...
4,S001203,<p><strong>Fair Pay for Federal Contractors Ac...
...,...,...
2746,M001233,<p>This joint resolution nullifies the final r...
2747,C001129,<p><strong>Laken Riley Act</strong></p><p>This...
2748,B001319,<p><strong>Laken Riley Act</strong></p><p>This...
2749,L000598,<p>This resolution disapproves of the Central ...


In [103]:
import re
def strip_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, ' ', text)

bill_description['text'] = bill_description['text'].apply(strip_html_tags)
bill_description


,bioguide_id,text
0,J000293,Shutdown Fairness Act This bill provides a...
1,O000173,Federal Worker Childcare Protection Act of 2...
2,A000380,This bill requires the federal government to ...
3,S001194,Federal Employees Civil Relief Act This b...
4,S001203,Fair Pay for Federal Contractors Act of 2025...
...,...,...
2746,M001233,This joint resolution nullifies the final rul...
2747,C001129,Laken Riley Act This bill requires the Dep...
2748,B001319,Laken Riley Act This bill requires the Dep...
2749,L000598,This resolution disapproves of the Central Bu...


In [104]:
bill_description = bill_description.groupby('bioguide_id').agg({'text': ' '.join}).reset_index()

In [105]:
x = ['have', 'a', 'nice', 'day']
x

['have', 'a', 'nice', 'day']

In [106]:
' '.join(x)

'have a nice day'

In [107]:
tfIdVectorizer = TfidfVectorizer(stop_words='english',
                                max_df = 0.8,
                                ngram_range = (1,3))
tfIdf = tfIdVectorizer.fit_transform(bill_description['text'])

In [108]:
tfIdf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 471845 stored elements and shape (490, 258655)>

## Take the top 10 by tf-idf per legislator

In [109]:
pd.DataFrame(tfIdf[1].T.todense(), index=tfIdVectorizer.get_feature_names_out(), 
             columns=["TF-IDF"]).sort_values(by=["TF-IDF"], ascending=False).head(10)

,TF-IDF
fringe benefits,0.236584
fringe,0.236584
transportation fringe benefits,0.168210
transportation fringe,0.157723
deduction exceed,0.157723
deduction,0.149925
employers,0.121866
transit,0.116094
transportation,0.105058
benefits,0.092505


In [110]:
def tfidf_one_legislator(index):
    print(index)
    tfidf_data = pd.DataFrame(tfIdf[index].T.todense(), index=tfIdVectorizer.get_feature_names_out(), 
                columns=["TF-IDF"]).sort_values(by=["TF-IDF"], ascending=False).head(10)
    tfidf_data['bioguide_id'] = bill_description['bioguide_id'][index]
    tfidf_data = tfidf_data.reset_index()
    tfidf_data = tfidf_data.rename({'index': 'keyword'}, axis=1)
    return tfidf_data

In [111]:
tfidf_one_legislator(100)

100


,keyword,TF-IDF,bioguide_id
0,advertising,0.135739,D000216
1,sampling,0.134147,D000216
2,fda,0.120953,D000216
3,cr,0.115368,D000216
4,drug,0.098234,D000216
5,consumer advertising,0.090810,D000216
6,direct consumer advertising,0.090810,D000216
7,animal feeding,0.090810,D000216
8,appropriations,0.088680,D000216
9,feeding,0.083203,D000216


In [112]:
tfidf_list = [tfidf_one_legislator(i) for i in range(len(bill_description))]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [113]:
tfidf_fulldata = pd.concat(tfidf_list)

In [114]:
tfidf_fulldata = tfidf_fulldata.rename({'TF-IDF': 'tf_idf'}, axis=1)

### Create a new table in the postgres DB with these charwords

In [115]:
tfidf_fulldata.to_csv('../data/tfidf_legislators.csv', index=False)

In [116]:
tfidf_fulldata.to_sql('tfidf', con=engine, index=False,
                      chunksize=1000, if_exists='replace')
                      

-5

### Check it for McGuire

In [117]:
myquery = '''
SELECT t.keyword, t.tf_idf
FROM tfidf t
INNER JOIN members m
    ON t.bioguide_id = m.bioguide_id
WHERE m.state_abbrev = 'VA' AND m.district_code = 5'''

pd.read_sql(myquery, con=engine)

,keyword,tf_idf
0,virginia,0.178550
1,dc,0.159192
2,highways,0.126302
3,agricultural vehicles,0.123226
4,covered agricultural,0.123226
5,agricultural,0.111179
6,interstate highways,0.105864
7,interstate,0.097627
8,vehicle,0.084542
9,commonly visited,0.082150
